In [1]:
# Importing useful libraries

import json
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import dask.dataframe as dd
import datetime

In [3]:
light_authors = "lighter_authors.json"
ch_size = 100
dfs = []

# to import only some columns
with open(light_authors, "r") as file:
    for ch in pd.read_json(file, lines = True, chunksize = ch_size):
        ch = ch[["name", "works_count","ratings_count","average_rating",
"text_reviews_count", "work_ids", "book_ids","id",
"gender",
"about","fans_count"]]
        dfs.append(ch)

la = pd.concat(dfs, ignore_index = True)

## [RQ7]  Estimating probabilities is a core skill for a data scientist: show us your best!

* Estimate the probability that a book has over 30% of the ratings above 4.

In [4]:
lighter_books = "lighter_books.json"
ch_size = 100
dfbs = []

# we import the dataset
with open(lighter_books, "r") as fileb:
    rows_read = 0
    for chb in pd.read_json(fileb, lines = True, chunksize = ch_size):
        chb = chb[["id", "title", "author_name","author_id", "work_id",
"language","average_rating","rating_dist","ratings_count","text_reviews_count","publication_date",
"original_publication_date", "format","edition_information","publisher","num_pages",
"series_id","series_name","series_position"]]
        dfbs.append(chb)
#       Update the count of rows read
        rows_read += len(chb)
        
        # Check if we have read 10,000 rows, and if so, break the loop
        if rows_read >= 100000:
            break
            
#final data with our books
lb = pd.concat(dfbs, ignore_index = True)


In [5]:
import pandas as pd
import re

#we chose the data we want to study, in this case lighter_books
dr = lb[['rating_dist','ratings_count']]

#we change rating_dist to str to be able to separate the values of each rating
dr['rating_dist'] = dr['rating_dist'].astype(str)


#we define a function to separate the ratings we want
def parse_rating_dist(rating_dist):
    match = re.search(r'5:(\d+)', rating_dist)
    if match:
        ratings_5 = int(match.group(1))
    else:
        ratings_5 = 0

    match = re.search(r'4:(\d+)', rating_dist)
    if match:
        ratings_4 = int(match.group(1))
    else:
        ratings_4 = 0  

    return ratings_4, ratings_5

# Apply the parsing function
dr['ratings_4'], dr['ratings_5'] = zip(*dr['rating_dist'].map(parse_rating_dist))



# Calculate the probability
dr['proportion_above_4'] = (dr['ratings_4'] + dr['ratings_5']) / dr['ratings_count']


# Estimate the probability
probability_above_30_percent = (dr['proportion_above_4'] > 0.30).mean()

print(probability_above_30_percent)

print(f"Estimated probability that a book has over 30% of ratings above 4: {probability_above_30_percent:.2%}")
pr = 0
for i in range(0,len(dr)):
    if dr['proportion_above_4'] [i] > 0.30:
        pr += 1
print('The probability to have over 30% of above 4 is',pr/(len(dr))*100,'%')


C:\Users\HP\AppData\Local\Temp\ipykernel_12344\3938496938.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dr['rating_dist'] = dr['rating_dist'].astype(str)
C:\Users\HP\AppData\Local\Temp\ipykernel_12344\3938496938.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dr['ratings_4'], dr['ratings_5'] = zip(*dr['rating_dist'].map(parse_rating_dist))
C:\Users\HP\AppData\Local\Temp\ipykernel_12344\3938496938.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

0.9565564925462612
Estimated probability that a book has over 30% of ratings above 4: 95.66%
The probability to have over 30% of above 4 is 95.65564925462611 %


* Estimate the probability that an author publishes a new book within two years from its last work.

In [6]:

import pandas as pd
from datetime import timedelta

# we get the columsn we'll need
dt = lb[['title','author_id','publication_date','original_publication_date']]

# we change the  datetime to be able touse it more easily
dt['original_publication_date'] = pd.to_datetime(dt['original_publication_date'], errors='coerce')

dt.sort_values(by=['author_id', 'original_publication_date'], ascending=True, inplace=True)
print(dt)
# Calculate time gaps
dt['time_gap'] = dt.groupby('author_id')['original_publication_date'].diff()
print(dt)

# Handle invalid (out-of-bounds) dates by replacing them with the previous work's publication date
dt['original_publication_date'] = dt.groupby('author_id')['original_publication_date'].fillna(method='ffill')
print(dt)

# Filter the data
filtered_data = dt.dropna(subset=['original_publication_date', 'time_gap'])
print(filtered_data)
filtered_data = filtered_data[filtered_data.groupby('author_id')['author_id'].transform('size') > 1]
print(filtered_data)
# Calculate the probability
probability_within_two_years = dt.groupby('author_id')['time_gap'].apply(lambda x: (x <= timedelta(days=730)).any()).mean()

print(f"Estimated probability that an author publishes a new book within two years from their last work: {probability_within_two_years:.2%}")


C:\Users\HP\AppData\Local\Temp\ipykernel_12344\61683674.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt['original_publication_date'] = pd.to_datetime(dt['original_publication_date'], errors='coerce')
C:\Users\HP\AppData\Local\Temp\ipykernel_12344\61683674.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt.sort_values(by=['author_id', 'original_publication_date'], ascending=True, inplace=True)


                                                   title  author_id  \
8      The Hitchhiker's Guide to the Galaxy (Hitchhik...          4   
11     The Hitchhiker's Guide to the Galaxy (Hitchhik...          4   
13     The Hitchhiker's Guide to the Galaxy (Hitchhik...          4   
2189   The Hitchhiker's Guide to the Galaxy (Hitchhik...          4   
5469   The Hitchhiker's Guide to the Galaxy (Hitchhik...          4   
...                                                  ...        ...   
91729                                              Rawls   20915206   
43245                        Momoko: A Novel of Betrayal   20915410   
44858               From Sea to Shining Sea for Children   20927297   
52776               The Light and the Glory for Children   20927297   
35033  Shakespeare's Plutarch: The Lives Julius Caesa...   20979420   

      publication_date original_publication_date  
8                 2005                1979-10-12  
11          2004-08-03                1979-10

OverflowError: Overflow in int64 addition

* In the file list.json, you will find a peculiar list named "The Worst Books of All Time." Estimate the probability of a book being included in this list, knowing it has more than 700 pages.

In [17]:
file_pathl = "list.json"
ch_size = 100
dfls = []

# to import only some columns
with open(file_pathl, "r") as filel:
    rows_read = 0
    for chl in pd.read_json(filel, lines = True, chunksize = ch_size):
        chl = chl[["id",
"title",
"description","description_html","num_pages",
"num_books",
"num_voters",
"created_date",
"tags",
"num_likes",
"created_by",
"num_comments",
"books"]]
        dfls.append(chl)
#          Update the count of rows read
        rows_read += len(chl)
        
        # Check if we have read 10,000 rows, and if so, break the loop
        if rows_read >= 100000:
            break

dfl = pd.concat(dfls, ignore_index = True)

In [49]:
import pandas as pd

lb['num_pages'] = pd.to_numeric(lb['num_pages'], errors='coerce')  # Convert to integers and handle errors
books_with_more_than_700_pages = lb[lb['num_pages'] > 700]

# Step 2: Identify books in "The Worst Books of All Time" list
# Assuming 'list' contains information about the list, extract the relevant book titles from it
rs = dfl[dfl['title'] == "The Worst Books of All Time"]

# Extract the titles of the books in the list

# Create a new Series containing book dictionaries
book_series = rs['books'].apply(lambda x: [item['title'] for item in x])

# Create a flattened list of titles
book_titles = [title for titles in book_series for title in titles]

# Step 2: Identify books in the extracted list
books_in_extracted_list = books_with_more_than_700_pages[books_with_more_than_700_pages['title'].isin(book_titles)]

# Step 3: Calculate the probability
total_books = len(books_with_more_than_700_pages)
books_in_list_count = len(books_in_extracted_list)

probability_in_list_given_700_pages = books_in_list_count / total_books

print(probability_in_list_given_700_pages)

0.16729323308270677


In [50]:
import pandas as pd

# Assuming 'lb' is a DataFrame containing book information
# 'list' is a DataFrame containing list information

# Step 1: Filter books with more than 700 pages from 'lb'
# Convert 'num_pages' to integers and then filter
lb['num_pages'] = pd.to_numeric(lb['num_pages'], errors='coerce')  # Convert to integers and handle errors
books_with_more_than_700_pages = lb[lb['num_pages'] > 700]

# Step 2: Identify books in "The Worst Books of All Time" list
# Assuming 'list' contains information about the list, extract the relevant book titles from it
worst_books_list = dfl[dfl['title'] == "The Worst Books of All Time"]

# Step 3: Calculate the probability
total_books = len(books_with_more_than_700_pages)
books_in_worst_list_count = len(worst_books_list)

probability_on_worst_list = books_in_worst_list_count / total_books

print(f"Estimated probability of a book being on 'The Worst Books of All Time' list, given it has more than 700 pages: {probability_on_worst_list:.2%}")


Estimated probability of a book being on 'The Worst Books of All Time' list, given it has more than 700 pages: 0.19%


In [52]:
# Calculate P(X), the probability of being in the list
total_books = len(books_with_more_than_700_pages)  # Number of books with more than 700 pages
books_in_worst_list_count = len(books_in_extracted_list)  # Number of books in the list

P_X = books_in_worst_list_count / total_books

# Calculate P(X|Y), the probability of being in the list given more than 700 pages
total_books_with_more_than_700_pages = len(books_with_more_than_700_pages)  # Total books with more than 700 pages
books_in_extracted_list_count = len(books_in_extracted_list)  # Number of books in the list with more than 700 pages
P_X_given_Y = books_in_extracted_list_count / total_books_with_more_than_700_pages

# Compare the probabilities
if P_X_given_Y == P_X:
    print("The events X and Y are independent.")
else:
    print("The events X and Y are dependent.")


The events X and Y are independent.
